In [1]:
%%writefile nltk_utils.py
import nltk
import numpy as np

# nltk.download('punkt')
# nltk.download('punkt_tab')





def tokenize(sentence):

    tokens = nltk.word_tokenize(sentence)


    return [w.lower() for w in tokens]

def encode_sentence(sentence, vocab_dict):
    encoded = []
    for word in sentence:
        if word in vocab_dict:
            encoded.append(vocab_dict[word])
        else:
            encoded.append(vocab_dict["<UNKN>"])
    return encoded

def pad_seq(batch, max_len):
    padded_batch = []
    for seq in batch:
        current_len = len(seq)
        if current_len < max_len:
            diff = max_len - current_len
            zeros = [0] * diff
            new_seq = seq + zeros
            padded_batch.append(new_seq)
        else:
            new_seq = seq[:max_len]
            padded_batch.append(new_seq)
    return padded_batch

Writing nltk_utils.py


In [2]:
%%writefile model.py
# model.py

import torch
import torch.nn as nn
class RNNNet(nn.Module):
  def __init__(self,vocab_size,embed_dim,hidden_size,num_classes):

    super(RNNNet,self).__init__()


    self.embed=nn.Embedding(vocab_size,embed_dim)



    self.lstm=nn.LSTM(embed_dim,hidden_size,batch_first=True)




    self.fc=nn.Linear(hidden_size,num_classes)




  def forward(self,x):


    out=self.embed(x)


    out,_=self.lstm(out)


    out=out[:,-1,:]



    out=self.fc(out)



    return out




Writing model.py


In [3]:
%%writefile train.py
import torch
import numpy as np
import torch.nn as nn
from nltk_utils import tokenize,encode_sentence,pad_seq
from model import RNNNet
from torch.utils.data import Dataset,DataLoader
import random
import json



with open('intents.json','r') as f:
  intents=json.load(f)


vocab={"<PAD>":0,"<UNKN>":1}
current_index=2

tags=[]
xy=[]

for intent in intents['intents']:
  tag=intent['tag']
  tags.append(tag)


  for pattern in intent['patterns']:
    w=tokenize(pattern)

    xy.append((w,tag))


    for word in w:
      if word not in vocab:
        vocab[word]=current_index
        current_index+=1





tags = sorted(set(tags))

max_len=10

X_train=[]
y_train=[]


for (pattern_sentence,tag) in xy:
  encoded_sentence=encode_sentence(pattern_sentence,vocab)
  X_train.append(encoded_sentence)
  label=tags.index(tag)


  y_train.append(label)


X_train=pad_seq(X_train,max_len)

X_train=np.array(X_train)
y_train=np.array(y_train)




class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples=len(X_train)
    self.x_data=X_train
    self.y_data=y_train

  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]

  def __len__(self):
    return self.n_samples





batch_size=8
hidden_size=32
embed_dim=16
output_size=len(tags)
vocab_size=len(vocab)
learning_rate=0.001
input_size = len(X_train[0])
num_epochs=2000



dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RNNNet(vocab_size, embed_dim, hidden_size, output_size).to(device)



criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)




for epoch in range(num_epochs):
    for (words, labels) in train_loader:


        words = words.to(dtype=torch.long).to(device)
        labels = labels.to(dtype=torch.long).to(device)


        outputs = model(words)


        loss = criterion(outputs, labels)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'Final Loss: {loss.item():.4f}')


data = {
    "model_state": model.state_dict(),
    "vocab_size": vocab_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "embed_dim": embed_dim,
    "vocab_dict": vocab,
    "tags": tags,
    "max_len": max_len
}

FILE = "data.pth"
torch.save(data, FILE)
print(f'Training complete. file saved to {FILE}')








Writing train.py


In [5]:
!python train.py

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Epoch [100/2000], Loss: 0.1522
Epoch [200/2000], Loss: 0.0269
Epoch [300/2000], Loss: 0.0076
Epoch [400/2000], Loss: 0.0067
Epoch [500/2000], Loss: 0.0019
Epoch [600/2000], Loss: 0.0014
Epoch [700/2000], Loss: 0.0007
Epoch [800/2000], Loss: 0.0005
Epoch [900/2000], Loss: 0.0002
Epoch [1000/2000], Loss: 0.0002
Epoch [1100/2000], Loss: 0.0001
Epoch [1200/2000], Loss: 0.0001
Epoch [1300/2000], Loss: 0.0000
Epoch [1400/2000], Loss: 0.0000
Epoch [1500/2000], Loss: 0.0000
Epoch [1600/2000], Loss: 0.0000
Epoch [1700/2000], Loss: 0.0000
Epoch [1800/2000], Loss: 0.0000
Epoch [1900/2000], Loss: 0.0000
Epoch [2000/2000], Loss: 0.0000
Final Loss: 0.0000
Training complete. file saved to data.pth


In [6]:
%%writefile chat.py
import json
import random
import torch
from nltk_utils import tokenize, encode_sentence, pad_seq
from model import RNNNet


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


with open('intents.json', 'r') as f:
    intents = json.load(f)

FILE = "data.pth"
data = torch.load(FILE)



vocab_size = data['vocab_size']

hidden_size = data['hidden_size']



output_size = data['output_size']
embed_dim = data['embed_dim']
vocab = data['vocab_dict']
tags = data['tags']
max_len = data['max_len']
model_state = data['model_state']




model = RNNNet(vocab_size, embed_dim, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()


bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")

while True:
    sentence = input("you: ")
    if sentence == "quit":
        break



    sentence_tokens = tokenize(sentence)
    encoded = encode_sentence(sentence_tokens, vocab)


    # print(f"DEBUG IDs: {encoded}")

    padded = pad_seq([encoded], max_len)




    X = torch.tensor(padded, dtype=torch.long).to(device)


    output = model(X)


    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]




    if prob.item() > 0.75:
        found_response = False
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
                found_response = True
                break


        if not found_response:
             print(f"{bot_name}: I do not understand... (Tag mismatch)")
    else:
        print(f"{bot_name}: I do not understand... (Low confidence)")

Writing chat.py


In [8]:
!python chat.py

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Let's chat! (type 'quit' to exit)
you: hello
Sam: Hi there! Nice to see you.
you: tell me a joke
Sam: Why did the developer go broke? Because he used up all his cache! 
you: what is your name
Sam: I am a Chatbot built with Python and PyTorch.
you: who is imran khan
Sam: He is the former PM and the 1992 World Cup winning captain.
you: what is cricket
Sam: It is a sport with 11 players per side.
you: tell me about pakistan
Sam: Pakistan is a beautiful country in South Asia, home to K2 and huge mountains.
you: what should i eat
Sam: Biryani is the answer to everything! 
you: ok thank you
Sam: Happy to help!
you: see you
Sam: Goodbye! Have a nice day 😊
you: quit
